In [1]:
import sys
sys.path.append('/workspaces/asclepius/src/Api')

print(sys.path)

['/workspaces/asclepius/src/Api/notebooks', '/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/home/vscode/.local/lib/python3.11/site-packages', '/usr/local/lib/python3.11/site-packages', '/workspaces/asclepius/src/Api']


In [2]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.kernel import Kernel, KernelArguments
from semantic_kernel.utils.settings import (
    azure_openai_settings_from_dot_env,
)
import pandas as pd
import tiktoken

import Plugins.parse_notes_plugin as pnp

In [3]:
enc = tiktoken.get_encoding('cl100k_base')

In [4]:
note_df = pd.read_csv('/workspaces/asclepius/src/Api/data/notes_no_PHI.csv')

# calculate length of 'NOTE_TEXT' column and add value as another column
note_df['NOTE_TEXT_TOKENS'] = note_df['NOTE_TEXT'].apply(lambda x: len(enc.encode(x)))

note_df['NOTE_TEXT_TOKENS']

0    1560
1    4203
Name: NOTE_TEXT_TOKENS, dtype: int64

## These native functions use only string parsing to fetch note sections

In [23]:
plugin = pnp.ParseNotesPlugin()

note_df['CHIEF_COMPLAINT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_chief_complaint(input=x))
note_df['CHIEF_COMPLAINT_TOKENS'] = note_df['CHIEF_COMPLAINT'].apply(lambda x: len(enc.encode(x)))

note_df['ROS'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_ros(input=x))
note_df['ROS_TOKENS'] = note_df['ROS'].apply(lambda x: len(enc.encode(x)))

note_df['ASSESSMENT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_assessment(input=x))
note_df['ASSESSMENT_TOKENS'] = note_df['ASSESSMENT'].apply(lambda x: len(enc.encode(x)))

note_df[['NOTE_TEXT_TOKENS', 'CHIEF_COMPLAINT_TOKENS', 'ROS_TOKENS', 'ASSESSMENT_TOKENS']]

#print(note_df['CHIEF_COMPLAINT'][1])


,NOTE_TEXT_TOKENS,CHIEF_COMPLAINT_TOKENS,ROS_TOKENS,ASSESSMENT_TOKENS
0,1560,481,0,447
1,4203,672,100,2624


In [ ]:
print(note_df['ASSESSMENT'][1])

## Use OpenAI to find and generate summaries of the note sections

In [15]:
deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
print(endpoint)

kernel = Kernel()
service_id = "default"

kernel.add_service(AzureChatCompletion(
    service_id=service_id,    
    deployment_name=deployment,
    endpoint=endpoint,
    api_key=api_key,
))

plugin = kernel.import_plugin_from_prompt_directory('/workspaces/asclepius/src/Api/Plugins', 'summarize_note')

functions = [
    plugin["summarize_chief_complaint"], 
    plugin["summarize_lab_history"],
    plugin["summarize_assessment"]
]

for idx, note in note_df.iterrows():
    arguments = KernelArguments(input=note["NOTE_TEXT"])
    response = await kernel.invoke(functions, arguments)

    note_df.loc[idx, 'SUM_CHIEF_COMPLAINT'] = response[0].value[0].content
    note_df.loc[idx, 'SUM_LAB_HISTORY'] = response[1].value[0].content
    note_df.loc[idx, 'SUM_ASSESSMENT'] = response[2].value[0].content
    
    note_df.loc[idx, 'SUM_CHIEF_COMPLAINT_TOKENS'] = int(len(enc.encode(response[0].value[0].content)))
    note_df.loc[idx, 'SUM_LAB_HISTORY_TOKENS'] = int(len(enc.encode(response[1].value[0].content)))
    note_df.loc[idx, 'SUM_ASSESSMENT_TOKENS'] = int(len(enc.encode(response[2].value[0].content)))

cols = ['NOTE_TEXT_TOKENS', 'SUM_CHIEF_COMPLAINT_TOKENS', 'SUM_LAB_HISTORY_TOKENS', 'SUM_ASSESSMENT_TOKENS']
note_df[cols]

#chief_complaint = response[0].value[0].content


https://phjirsa-aoai.openai.azure.com/


,NOTE_TEXT_TOKENS,SUM_CHIEF_COMPLAINT_TOKENS,SUM_LAB_HISTORY_TOKENS,SUM_ASSESSMENT_TOKENS
0,1560,214.0,14.0,242.0
1,4203,264.0,203.0,201.0


In [44]:
note_df[cols].to_csv('/workspaces/asclepius/src/Api/data/output/note_summary.csv', index=False)

In [17]:
print(note_df['SUM_CHIEF_COMPLAINT'][0])

The patient, Sky Ryder, is a 27-year-old female who presented to establish care and for a physical examination with annual labs. She has a history of chronic pain in her right knee and had surgery in 2016 but continues to experience pain. She would like a referral to an orthopedic specialist. She is currently taking celexa daily for depression and is doing well on it. She is also on semaglutide through a weight loss clinic and is doing well with that. Previous labs showed low HDL and elevated CRP. She is also being seen for follow-up of hypothyroidism, but denies any symptoms related to it. Her current medications include citalopram, folic acid, levothyroxine, and semaglutide. The patient has no significant findings on physical examination and her vital signs are within normal limits. The plan includes a referral to ortho for her knee pain, alternating ibuprofen and Tylenol for pain relief, and scheduling with a gynecologist for a PAP smear.


In [14]:
note_df['NOTE_TEXT'][1]

"Depression Screen \nPositive:  PHQ-2 score >= 3 or PHQ-9 score >= 9\nPHQ-2 Total: 0 (4/16/2024 11:35 AM)\nPHQ-9 Total: 0 (4/16/2024 11:35 AM)\n \nDEPRESSION PLAN OF CARE\nHer depression screen was negative.\n \nChase Humdinger is a 83 y.o. female\n\nChief Complaint\nPatient presents with\n• Annual Wellness Visit (Medicare)\n• Medication Review\n  Patient has been taking half of all her medications, patient states her smart watch tells her that she has a fib\n \n \nPt is here for routine check up.\nShe has history of palpitations and states occasionally she will be notified by her watch that her pulse is high while she feels the palpitations. She has cut the cytomel in half and is taking over other day which she feels like has improved her symptoms. \nShe's also been cutting her toprol in half and BP has been high. Discussed taking full pull routinely to help regulate pulse and improve BP. \n \nReviewed previous labs \nHistory of elevated prolactin - pt didn't like the medication presc